#  <font color='blue'>Mean Machine: Decision Tree Part 3</font>

In this notebook, you will explore various techniques for preventing overfitting in decision trees. The tasks herewith are to

* Implement binary decision trees with different early stopping methods.
* Compare models with different stopping parameters.
* Visualize the concept of overfitting in decision trees.

# <font color='red'>Import all relevant packages</font>

In [1]:
import numpy as np     # 用来做数学运算
import pandas as pd    # 用来处理数据表
from sklearn.model_selection import train_test_split # 做交叉验证，划分训练集和测试集

# <font color='red'>Load Lending Club dataset</font>

We will use the same [LendingClub](https://www.lendingclub.com/) dataset as in the previous notebook.

In [2]:
loans = pd.read_csv('lending-club-data.csv', low_memory=False)
loans.head(3).append(loans.tail(3))

,id,member_id,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,grade,sub_grade,...,sub_grade_num,delinq_2yrs_zero,pub_rec_zero,collections_12_mths_zero,short_emp,payment_inc_ratio,final_d,last_delinq_none,last_record_none,last_major_derog_none
0,1077501,1296599,5000,5000,4975,36 months,10.65,162.87,B,B2,...,0.4,1.0,1.0,1.0,0,8.143500,20141201T000000,1,1,1
1,1077430,1314167,2500,2500,2500,60 months,15.27,59.83,C,C4,...,0.8,1.0,1.0,1.0,1,2.393200,20161201T000000,1,1,1
2,1077175,1313524,2400,2400,2400,36 months,15.96,84.33,C,C5,...,1.0,1.0,1.0,1.0,0,8.259550,20141201T000000,1,1,1
122604,9695736,11547808,8525,8525,8525,60 months,18.25,217.65,D,D3,...,0.6,0.0,1.0,1.0,0,6.958120,20190101T000000,0,1,0
122605,9684700,11536848,22000,22000,22000,60 months,19.97,582.50,D,D5,...,1.0,1.0,0.0,1.0,0,8.961540,20190101T000000,1,0,1
122606,9604874,11457002,2000,2000,2000,36 months,7.90,62.59,A,A4,...,0.8,0.0,1.0,1.0,0,0.904916,20170101T000000,0,1,1


Reassign the labels to have +1 for a safe loan, and -1 for a risky (bad) loan.

In [3]:
loans['safe_loans'] = loans['bad_loans'].apply(lambda x : +1 if x==0 else -1)
loans = loans.drop('bad_loans', 1)

In this notebook, we will just use 4 categorical features: 

1. grade of the loan 
2. the length of the loan term
3. the home ownership status: own, mortgage, rent
4. number of years of employment.

Since we are building a binary decision tree, we will have to convert these categorical features to a binary representation in a subsequent section using 1-hot encoding.

In [4]:
features = ['grade',              # grade of the loan
            'term',               # the term of the loan
            'home_ownership',     # home_ownership status: own, mortgage or rent
            'emp_length',         # number of years of employment
           ]
target = 'safe_loans'
loans = loans[features + [target]]

## Subsample dataset to make sure classes are balanced

We will undersample the larger class (safe loans) in order to balance out our dataset. This means we are throwing away many data points. We use `seed=1` so everyone gets the same results.

In [5]:
safe_loans_raw = loans[loans[target] == +1]
risky_loans_raw = loans[loans[target] == -1]

ratio = len(risky_loans_raw)/float(len(safe_loans_raw))

risky_loans = risky_loans_raw
safe_loans = safe_loans_raw.sample(frac=ratio, random_state=1)

# Append the risky_loans with the downsampled version of safe_loans
loans_data = risky_loans.append(safe_loans)

N1 = len(safe_loans)
N2 = len(risky_loans)
N = N1 + N2
print( "%% of safe loans  : %.2f%%" %(N1/N*100.0) )
print( "%% of risky loans : %.2f%%" %(N2/N*100.0) )
print( "Total number of loans in our new dataset :", N )

% of safe loans  : 50.00%
% of risky loans : 50.00%
Total number of loans in our new dataset : 46300


## Transform categorical data into binary features

We will implement **binary decision trees** (decision trees for binary features, a specific case of categorical variables taking on two values, e.g., true/false). Since all of our features are currently categorical features, we want to turn them into binary features. 

For instance, the **home_ownership** feature represents the home ownership status of the loanee, which is either `own`, `mortgage` or `rent`. For example, if a data point has the feature 
```
   {'home_ownership': 'RENT'}
```
we want to turn this into three features: 
```
 { 
   'home_ownership = OWN'      : 0, 
   'home_ownership = MORTGAGE' : 0, 
   'home_ownership = RENT'     : 1
 }
```

In [6]:
categorical_variables = []
for feat_name, feat_type in zip(loans_data.columns.values,loans_data.dtypes):
    if feat_type == object:
        categorical_variables.append(feat_name)

for feature in categorical_variables:
    feat_value = loans_data[feature].unique()
    loans_data_one_hot_encoded = pd.DataFrame()
    for val in feat_value:
        label = feature + '.' + val
        loans_data_one_hot_encoded[label] = loans_data[feature].apply(lambda x: 1 if x == val else 0)
    loans_data = pd.concat([loans_data, loans_data_one_hot_encoded], axis=1)
loans_data = loans_data.drop(categorical_variables,axis=1)

loans_data.head(3).append(loans_data.tail(3))

,safe_loans,grade.C,grade.F,grade.B,grade.D,grade.A,grade.E,grade.G,term. 60 months,term. 36 months,...,emp_length.3 years,emp_length.10+ years,emp_length.1 year,emp_length.9 years,emp_length.2 years,emp_length.8 years,emp_length.7 years,emp_length.5 years,emp_length.n/a,emp_length.6 years
1,-1,1,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
6,-1,0,1,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
7,-1,0,0,1,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
90431,1,0,0,1,0,0,0,0,0,1,...,0,0,0,0,0,1,0,0,0,0
115727,1,0,1,0,0,0,0,0,1,0,...,1,0,0,0,0,0,0,0,0,0
105752,1,0,0,0,1,0,0,0,1,0,...,0,0,1,0,0,0,0,0,0,0


The feature columns now look like this:

In [7]:
features = loans_data.columns.values
features = features[features != target]
features

array(['grade.C', 'grade.F', 'grade.B', 'grade.D', 'grade.A', 'grade.E',
       'grade.G', 'term. 60 months', 'term. 36 months',
       'home_ownership.RENT', 'home_ownership.OWN',
       'home_ownership.MORTGAGE', 'home_ownership.OTHER',
       'emp_length.< 1 year', 'emp_length.4 years', 'emp_length.3 years',
       'emp_length.10+ years', 'emp_length.1 year', 'emp_length.9 years',
       'emp_length.2 years', 'emp_length.8 years', 'emp_length.7 years',
       'emp_length.5 years', 'emp_length.n/a', 'emp_length.6 years'], dtype=object)

## Train-Validation split

We split the data into a train-validation split with 80% of the data in the training set and 20% of the data in the validation set. We use `random_state=0` so that everyone gets the same result.

In [8]:
(train_data, validation_data) = train_test_split( loans_data, 
                             train_size=0.8, random_state=0 )
print( train_data.shape, validation_data.shape )

(37040, 26) (9260, 26)


# <font color='red'>Early stopping methods for decision trees</font>

In this section, we will extend the **binary tree implementation** from the previous assignment in order to handle some early stopping conditions. The 3 early stopping methods are:

1. Reached a **maximum depth**. (set by parameter `max_depth`).
2. Reached a **minimum node size**. (set by parameter `min_node_size`).
3. Don't split if the **gain in error reduction** is too small. (set by parameter `min_error_reduction`).

Next, we will refer to these three as **early stopping conditions 1, 2, and 3**.

## <font color='green'>Early stopping condition 1: Maximum depth</font>

We will experiment with this condition a bit more and also write code to implement the 2nd and 3rd early stopping conditions.

## <font color='green'>Early stopping condition 2: Minimum node size</font>

The function **reached_minimum_node_size** takes 2 arguments:

1. The `data` (from a node)
2. The minimum number of data points that a node is allowed to split on, `min_node_size`.

This function simply calculates whether the number of data points at a given node is less than or equal to the specified minimum node size. This function will be used to detect this early stopping condition in the **decision_tree_create** function.

In [9]:
def reached_minimum_node_size( data, min_node_size ):
    return len(data) <= min_node_size

## <font color='green'>Early stopping condition 3: Minimum gain in error reduction</font>

The function **error_reduction** takes 2 arguments:

1. The error **before** a split, `error_before_split`.
2. The error **after** a split, `error_after_split`.

This function computes the gain in error reduction, i.e., the difference between the error before the split and that after the split. This function will be used to detect this early stopping condition in the **decision_tree_create** function. 

In [10]:
def error_reduction( error_before_split, error_after_split ):
    return error_before_split - error_after_split

Recall from the previous notebook that we wrote a function `count_num_mistakes` that calculates the number of **misclassified examples** when predicting the **majority class**. This is used to help determine which feature is best to split on at a given node of the tree.

In [11]:
def count_num_mistakes( labels_in_node ):
    # Corner case: If labels_in_node is empty, return 0
    if len(labels_in_node) == 0:
        return 0
    
    # Count the number of 1's (safe loans)
    counter1 = (labels_in_node == 1).sum()
    
    # Count the number of -1's (risky loans)
    counter_1 = (labels_in_node == -1).sum()
                
    # Return the number of mistakes by majority rules
    return counter_1 if counter1 >= counter_1 else counter1 

We then wrote a function `best_splitting_feature` that finds the best feature to split on given the data and a list of features to consider.

In [12]:
def best_splitting_feature( data, features, target ):
    
    best_feature = None # Keep track of the best feature 
    best_error = 10     # Keep track of the best error so far 
    # Note: Since error is always <= 1 so it's OK with some number > 1.

    # Convert to float to make sure error gets computed correctly.
    num_data_points = float(len(data))  
    
    # Loop through each feature to consider splitting on that feature
    for feature in features:
        
        # The left split will have all data points where the feature value is 0
        left_split = data[ data[feature] == 0 ]
        
        # The right split will have all data points where the feature value is 1
        right_split = data[ data[feature] == 1 ] 
            
        # Calculate the number of misclassified examples in the left split.
        left_mistakes = count_num_mistakes( left_split[target] )            

        # Calculate the number of misclassified examples in the right split.
        right_mistakes = count_num_mistakes( right_split[target] ) 
            
        # Compute the classification error of this split.
        # Error = (# of mistakes (left) + # of mistakes (right)) / (# of data points)
        error = (left_mistakes + right_mistakes) / num_data_points

        # If this is the best error we have found so far, 
        # store the feature as best_feature and the error as best_error
        if error < best_error:
            best_feature = feature
            best_error = error
    
    return best_feature # Return the best feature we found

Finally, recall the function `create_leaf` from the previous notebook, which creates a leaf node given a set of target values.  

In [13]:
def create_leaf( target_values ):

    leaf = {'splitting_feature' : None,
            'left' : None,
            'right' : None,
            'is_leaf': True }
    
    # Count the number of data points that are +1 and -1 in this node.
    num_ones = len( target_values[target_values == +1] )
    num_minus_ones = len( target_values[target_values == -1] )
    
    # For the leaf node, set the prediction to be the majority class.
    # Store the predicted class (1 or -1) in leaf['prediction']
    leaf['prediction'] = 1 if num_ones > num_minus_ones else -1
     
    return leaf

## Incorporating new early stopping conditions in binary decision tree implementation

Now, you will implement a function that builds a decision tree handling the three early stopping conditions described above.

**Implementing early stopping condition 2: minimum node size:**

* **Step 1:** Use the function **reached_minimum_node_size** that you implemented earlier to write an if condition to detect whether we have hit the base case, i.e., the node does not have enough data points and should be turned into a leaf. Don't forget to use the `min_node_size` argument.
* **Step 2:** Return a leaf.

**Implementing early stopping condition 3: minimum error reduction:**

**Note:** This has to come after finding the best splitting feature so we can calculate the error after splitting in order to calculate the error reduction.

* **Step 1:** Calculate the **classification error before splitting**.  Recall that classification error is defined as:

$$
\text{classification error} = \frac{\text{# mistakes}}{\text{# total examples}}
$$
* **Step 2:** Calculate the **classification error after splitting**. This requires calculating the number of mistakes in the left and right splits, and then dividing by the total number of examples.
* **Step 3:** Use the function **error_reduction** to that you implemented earlier to write an if condition to detect whether  the reduction in error is less than the constant provided (`min_error_reduction`). Don't forget to use that argument.
* **Step 4:** Return a leaf.

In [19]:
def decision_tree( data, features, target, current_depth = 0, max_depth = 10, 
                   min_node_size=1,  min_error_reduction=0.0 ):
    
    remaining_features = features[:] # Make a copy of the features.
    
    target_values = data[target]
    print( "--------------------------------------------------------------------" )
    print( "Subtree, depth = %s (%s data points)." % (current_depth, len(target_values)) )
    
    # Stopping condition 1: All nodes are of the same type.
    if count_num_mistakes(target_values) == 0:
        print( "Stopping condition 1 reached. All data points have the same target value." )                
        return create_leaf(target_values)
    
    # Stopping condition 2: No more features to split on.
    if len(remaining_features) == 0:
        print( "Stopping condition 2 reached. No remaining features." )              
        return create_leaf(target_values)    
    
    # Early stopping condition 1: Reached max depth limit.
    if current_depth >= max_depth:
        print( "Early stopping condition 1 reached. Reached maximum depth." )
        return create_leaf(target_values)
    
    # Early stopping condition 2: Reached the minimum node size.
    # If the number of data points <= the minimum size, return a leaf.
    if reached_minimum_node_size(data, min_node_size): 
        print( "Early stopping condition 2 reached. Reached minimum node size." )
        return create_leaf(target_values)
    
    # Find the best splitting feature
    splitting_feature = best_splitting_feature(data, features, target)
    
    # Split on the best feature that we found. 
    left_split = data[data[splitting_feature] == 0]
    right_split = data[data[splitting_feature] == 1]
    
    # Early stopping condition 3: Minimum error reduction
    # Calculate the error before splitting (number of misclassified examples 
    # divided by the total number of examples)
    error_before_split = count_num_mistakes(target_values) / float(len(data))
    
    # Calculate the error after splitting (number of misclassified examples 
    # in both groups divided by the total number of examples)
    left_mistakes = count_num_mistakes(left_split[target])
    right_mistakes = count_num_mistakes(right_split[target])
    error_after_split = (left_mistakes + right_mistakes) / float(len(data))
    
    # If the error reduction <= min_error_reduction, return a leaf.
    if error_reduction(error_before_split, error_after_split) <= min_error_reduction:
        print( "Early stopping condition 3 reached. Minimum error reduction." )
        return create_leaf(target_values) 
    
    remaining_features = remaining_features[remaining_features != splitting_feature]
    print( "Split on feature %s. (%s, %s)" % (\
                      splitting_feature, len(left_split), len(right_split)) )
    
    # Repeat on left and right subtrees
    left_tree = decision_tree( left_split, remaining_features, target, current_depth + 1, 
                               max_depth, min_node_size, min_error_reduction )        
    right_tree = decision_tree( right_split, remaining_features, target, current_depth + 1, 
                                max_depth, min_node_size, min_error_reduction ) 
    
    return {'is_leaf'          : False, 
            'prediction'       : None,
            'splitting_feature': splitting_feature,
            'left'             : left_tree, 
            'right'            : right_tree}

Here is a function to count the nodes in your tree:

In [15]:
def count_nodes(tree):
    if tree['is_leaf']:
        return 1
    return 1 + count_nodes(tree['left']) + count_nodes(tree['right'])

Run the following test code to check your implementation. Make sure you get **'Test passed'** before proceeding.

In [20]:
small_decision_tree = decision_tree( train_data, features, 'safe_loans', max_depth = 2, 
                                     min_node_size = 10, min_error_reduction=0.0 )
if count_nodes(small_decision_tree) == 7:
    print( 'Test passed!' )
else:
    print( 'Test failed... try again!' )
    print( 'Number of nodes found                :', count_nodes(small_decision_tree) )
    print( 'Number of nodes that should be there : 7' )

--------------------------------------------------------------------
Subtree, depth = 0 (37040 data points).
Split on feature term. 60 months. (27703, 9337)
--------------------------------------------------------------------
Subtree, depth = 1 (27703 data points).
Split on feature grade.D. (23068, 4635)
--------------------------------------------------------------------
Subtree, depth = 2 (23068 data points).
Early stopping condition 1 reached. Reached maximum depth.
--------------------------------------------------------------------
Subtree, depth = 2 (4635 data points).
Early stopping condition 1 reached. Reached maximum depth.
--------------------------------------------------------------------
Subtree, depth = 1 (9337 data points).
Split on feature grade.A. (9216, 121)
--------------------------------------------------------------------
Subtree, depth = 2 (9216 data points).
Early stopping condition 1 reached. Reached maximum depth.
----------------------------------------------

## Build a tree!

Now that your code is working, we will train a tree model on the **train_data** with
* `max_depth = 6`
* `min_node_size = 100`, 
* `min_error_reduction = 0.0`

In [21]:
my_decision_tree_new = decision_tree( train_data, features, 'safe_loans', max_depth = 6, 
                                      min_node_size = 100, min_error_reduction=0.0 )

--------------------------------------------------------------------
Subtree, depth = 0 (37040 data points).
Split on feature term. 60 months. (27703, 9337)
--------------------------------------------------------------------
Subtree, depth = 1 (27703 data points).
Split on feature grade.D. (23068, 4635)
--------------------------------------------------------------------
Subtree, depth = 2 (23068 data points).
Split on feature grade.E. (21809, 1259)
--------------------------------------------------------------------
Subtree, depth = 3 (21809 data points).
Split on feature grade.C. (14639, 7170)
--------------------------------------------------------------------
Subtree, depth = 4 (14639 data points).
Split on feature grade.F. (14265, 374)
--------------------------------------------------------------------
Subtree, depth = 5 (14265 data points).
Split on feature grade.G. (14164, 101)
--------------------------------------------------------------------
Subtree, depth = 6 (14164 data 

Let's now train a tree model **ignoring early stopping conditions 2 and 3** so that we get the same tree as in the previous notebook.  To ignore these conditions, we set `min_node_size=0` and `min_error_reduction=-1` (a negative value).

In [22]:
my_decision_tree_old = decision_tree( train_data, features, 'safe_loans', max_depth = 6, 
                                      min_node_size = 0, min_error_reduction=-1 )

--------------------------------------------------------------------
Subtree, depth = 0 (37040 data points).
Split on feature term. 60 months. (27703, 9337)
--------------------------------------------------------------------
Subtree, depth = 1 (27703 data points).
Split on feature grade.D. (23068, 4635)
--------------------------------------------------------------------
Subtree, depth = 2 (23068 data points).
Split on feature grade.E. (21809, 1259)
--------------------------------------------------------------------
Subtree, depth = 3 (21809 data points).
Split on feature grade.C. (14639, 7170)
--------------------------------------------------------------------
Subtree, depth = 4 (14639 data points).
Split on feature grade.F. (14265, 374)
--------------------------------------------------------------------
Subtree, depth = 5 (14265 data points).
Split on feature grade.G. (14164, 101)
--------------------------------------------------------------------
Subtree, depth = 6 (14164 data 

## Making predictions

Recall that in the previous notebook you implemented a function `classify` to classify a new point `x` using a given `tree`.

In [23]:
def classify( tree, x, annotate = False ):   
    # if the node is a leaf node.
    if tree['is_leaf']:
        if annotate: 
            print( "At leaf, predicting %s" % tree['prediction'] )
        return tree['prediction'] 
    else:
        # split on feature.
        split_feature_value = x[tree['splitting_feature']]
        if annotate: 
            print( "Split on %s = %s" % (tree['splitting_feature'], split_feature_value) )
        if split_feature_value == 0:
            return classify( tree['left'], x, annotate )
        else:
            return classify( tree['right'], x, annotate )

Now, let's consider the first example of the validation set and see what the `my_decision_tree_new` and `my_decision_tree_old` model predicts for this data point.

In [24]:
validation_data.iloc[0]

safe_loans                -1
grade.C                    0
grade.F                    0
grade.B                    0
grade.D                    0
grade.A                    0
grade.E                    1
grade.G                    0
term. 60 months            1
term. 36 months            0
home_ownership.RENT        0
home_ownership.OWN         1
home_ownership.MORTGAGE    0
home_ownership.OTHER       0
emp_length.< 1 year        0
emp_length.4 years         0
emp_length.3 years         1
emp_length.10+ years       0
emp_length.1 year          0
emp_length.9 years         0
emp_length.2 years         0
emp_length.8 years         0
emp_length.7 years         0
emp_length.5 years         0
emp_length.n/a             0
emp_length.6 years         0
Name: 16626, dtype: int64

In [28]:
classify( my_decision_tree_new, validation_data.iloc[0], annotate = True )
print('------------------------------')
classify( my_decision_tree_old, validation_data.iloc[0], annotate = True )

Split on term. 60 months = 1
Split on grade.A = 0
At leaf, predicting -1
------------------------------
Split on term. 60 months = 1
Split on grade.A = 0
Split on grade.C = 0
Split on grade.F = 0
Split on grade.B = 0
Split on grade.D = 0
At leaf, predicting -1


-1

## Evaluating the model

Now let us evaluate the model that we have trained. You implemented this evautation in the function `misclassify_error` from the previous assignment.

In [29]:
def misclassify_error( tree, data ):
    # Apply the classify(tree, x) to each row in your data
    prediction = data.apply( lambda x: classify(tree, x), axis=1 )
    true_label = data["safe_loans"]
    return (prediction!=true_label).sum() / float(len(prediction))

Now, let's use this function to evaluate the classification error of `my_decision_tree_new` and `my_decision_tree_old` on the **validation_set**.

In [31]:
misclassify_error( my_decision_tree_new, validation_data )

0.39092872570194387

In [29]:
misclassify_error( my_decision_tree_old, validation_data )

0.39146868250539957

# <font color='red'>Exploring the effect of max_depth</font>

We will compare three models trained with different values of the stopping criterion. We intentionally picked models at the extreme ends (**too small**, **just right**, and **too large**).

Train three models with these parameters:

1. **model_1**: max_depth = 2 (too small)
2. **model_2**: max_depth = 6 (just right)
3. **model_3**: max_depth = 14 (may be too large)

For each of these three, we set `min_node_size = 0` and `min_error_reduction = -1`.

In [32]:
model_1 = decision_tree( train_data, features, 'safe_loans', max_depth = 2, 
                         min_node_size = 0, min_error_reduction=-1 )
model_2 = my_decision_tree_old
model_3 = decision_tree( train_data, features, 'safe_loans', max_depth = 14, 
                         min_node_size = 0, min_error_reduction=-1 )

--------------------------------------------------------------------
Subtree, depth = 0 (37040 data points).
Split on feature term. 60 months. (27703, 9337)
--------------------------------------------------------------------
Subtree, depth = 1 (27703 data points).
Split on feature grade.D. (23068, 4635)
--------------------------------------------------------------------
Subtree, depth = 2 (23068 data points).
Early stopping condition 1 reached. Reached maximum depth.
--------------------------------------------------------------------
Subtree, depth = 2 (4635 data points).
Early stopping condition 1 reached. Reached maximum depth.
--------------------------------------------------------------------
Subtree, depth = 1 (9337 data points).
Split on feature grade.A. (9216, 121)
--------------------------------------------------------------------
Subtree, depth = 2 (9216 data points).
Early stopping condition 1 reached. Reached maximum depth.
----------------------------------------------

### Evaluating the models

Let us evaluate the models on the **training** and **validation** data. Let us evaluate the classification error on the **training data** and **validation data**

In [31]:
print( "Training data, classification error (model 1):", misclassify_error(model_1, train_data) )
print( "Training data, classification error (model 2):", misclassify_error(model_2, train_data) )
print( "Training data, classification error (model 3):", misclassify_error(model_3, train_data) )

Training data, classification error (model 1): 0.401430885529
Training data, classification error (model 2): 0.379454643629
Training data, classification error (model 3): 0.375215982721


In [33]:
print( "Validation data, classification error (model 1):", misclassify_error(model_1, validation_data) )
print( "Validation data, classification error (model 2):", misclassify_error(model_2, validation_data) )
print( "Validation data, classification error (model 3):", misclassify_error(model_3, validation_data) )

Validation data, classification error (model 1): 0.405939524838
Validation data, classification error (model 2): 0.391468682505
Validation data, classification error (model 3): 0.394060475162


### Measuring the complexity of the tree

Recall that deeper trees are more complex. We will measure the complexity of the tree as

```
  complexity(T) = number of leaves in the tree T
```

Here, we provide a function `count_leaves` that counts the number of leaves in a tree. Using this implementation, compute the number of nodes in `model_1`, `model_2`, and `model_3`. 

In [34]:
def count_leaves(tree):
    if tree['is_leaf']:
        return 1
    return count_leaves(tree['left']) + count_leaves(tree['right'])

Compute the number of nodes in `model_1`, `model_2`, and `model_3`.

In [35]:
print( "Number of leaves (model 1):", count_leaves(model_1) )
print( "Number of leaves (model 2):", count_leaves(model_2) )
print( "Number of leaves (model 3):", count_leaves(model_3) )

Number of leaves (model 1): 4
Number of leaves (model 2): 38
Number of leaves (model 3): 393


# <font color='red'>Exploring the effect of min_error</font>

We will compare three models trained with different values of the stopping criterion. We intentionally picked models at the extreme ends (**negative**, **just right**, and **too positive**).

Train three models with these parameters:
1. **model_4**: `min_error_reduction = -1` (ignoring this early stopping condition)
2. **model_5**: `min_error_reduction = 0` (just right)
3. **model_6**: `min_error_reduction = 5` (too positive)

For each of these three, we set `max_depth = 6`, and `min_node_size = 0`.

In [36]:
model_4 = model_2
model_5 = decision_tree( train_data, features, 'safe_loans', max_depth = 6, 
                                min_node_size = 0, min_error_reduction=0 )
model_6 = decision_tree( train_data, features, 'safe_loans', max_depth = 6, 
                                min_node_size = 0, min_error_reduction=5 )

--------------------------------------------------------------------
Subtree, depth = 0 (37040 data points).
Split on feature term. 60 months. (27703, 9337)
--------------------------------------------------------------------
Subtree, depth = 1 (27703 data points).
Split on feature grade.D. (23068, 4635)
--------------------------------------------------------------------
Subtree, depth = 2 (23068 data points).
Split on feature grade.E. (21809, 1259)
--------------------------------------------------------------------
Subtree, depth = 3 (21809 data points).
Split on feature grade.C. (14639, 7170)
--------------------------------------------------------------------
Subtree, depth = 4 (14639 data points).
Split on feature grade.F. (14265, 374)
--------------------------------------------------------------------
Subtree, depth = 5 (14265 data points).
Split on feature grade.G. (14164, 101)
--------------------------------------------------------------------
Subtree, depth = 6 (14164 data 

Calculate the accuracy of each model (**model_4**, **model_5**, or **model_6**) on the **validation data**. 

In [37]:
print( "Validation data, classification error (model 4):", misclassify_error(model_4, validation_data) )
print( "Validation data, classification error (model 5):", misclassify_error(model_5, validation_data) )
print( "Validation data, classification error (model 6):", misclassify_error(model_6, validation_data) )

Validation data, classification error (model 4): 0.391468682505
Validation data, classification error (model 5): 0.390928725702
Validation data, classification error (model 6): 0.503455723542


In [38]:
print( "Number of leaves (model 4):", count_leaves(model_4) )
print( "Number of leaves (model 5):", count_leaves(model_5) )
print( "Number of leaves (model 6):", count_leaves(model_6) )

Number of leaves (model 4): 38
Number of leaves (model 5): 13
Number of leaves (model 6): 1


**Quiz Question:** Using the complexity definition above, which model (**model_4**, **model_5**, or **model_6**) has the largest complexity?

Did this match your expectation?

**Quiz Question:** **model_4** and **model_5** have similar classification error on the validation set but **model_5** has lower complexity? Should you pick **model_5** over **model_4**?


# <font color='red'>Exploring the effect of min_node_size</font>

We will compare three models trained with different values of the stopping criterion. Again, intentionally picked models at the extreme ends (**too small**, **just right**, and **just right**).

Train three models with these parameters:
1. **model_7**: min_node_size = 0 (too small)
2. **model_8**: min_node_size = 2000 (just right)
3. **model_9**: min_node_size = 50000 (too large)

For each of these three, we set `max_depth = 6`, and `min_error_reduction = -1`.

In [39]:
model_7 = model_2
model_8 = decision_tree( train_data, features, 'safe_loans', max_depth = 6, 
                         min_node_size = 2000, min_error_reduction=-1 )
model_9 = decision_tree( train_data, features, 'safe_loans', max_depth = 6, 
                         min_node_size = 50000, min_error_reduction=-1 )

--------------------------------------------------------------------
Subtree, depth = 0 (37040 data points).
Split on feature term. 60 months. (27703, 9337)
--------------------------------------------------------------------
Subtree, depth = 1 (27703 data points).
Split on feature grade.D. (23068, 4635)
--------------------------------------------------------------------
Subtree, depth = 2 (23068 data points).
Split on feature grade.E. (21809, 1259)
--------------------------------------------------------------------
Subtree, depth = 3 (21809 data points).
Split on feature grade.C. (14639, 7170)
--------------------------------------------------------------------
Subtree, depth = 4 (14639 data points).
Split on feature grade.F. (14265, 374)
--------------------------------------------------------------------
Subtree, depth = 5 (14265 data points).
Split on feature grade.G. (14164, 101)
--------------------------------------------------------------------
Subtree, depth = 6 (14164 data 

Now, let us evaluate the models (**model_7**, **model_8**, or **model_9**) on the **validation_data**.

In [41]:
print( "Validation data, classification error (model 7):", misclassify_error(model_7, validation_data) )
print( "Validation data, classification error (model 8):", misclassify_error(model_8, validation_data) )
print( "Validation data, classification error (model 9):", misclassify_error(model_9, validation_data) )

Validation data, classification error (model 7): 0.391468682505
Validation data, classification error (model 8): 0.390712742981
Validation data, classification error (model 9): 0.503455723542


In [42]:
print( "Number of leaves (model 7):", count_leaves(model_7) )
print( "Number of leaves (model 8):", count_leaves(model_8) )
print( "Number of leaves (model 9):", count_leaves(model_9) )

Number of leaves (model 7): 38
Number of leaves (model 8): 22
Number of leaves (model 9): 1
